# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'weatherpy_cities.csv')

cities_df = pd.read_csv(csv_file)
# cities_df.count()

cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,0,Kariba,ZW,-16.5167,28.8000,84.11,44,52,17.38,2021-10-18
1,1,Puro,PH,12.4841,123.3827,79.97,87,62,6.89,2021-10-18
2,2,Mizdah,LY,31.4451,12.9801,69.91,48,100,15.14,2021-10-18
3,3,Dicabisagan,PH,17.0818,122.4157,77.90,87,71,3.94,2021-10-18
4,4,Ponta do Sol,PT,32.6667,-17.1000,79.12,34,0,2.86,2021-10-18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Convert to float
humidity = cities_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities_df = cities_df.loc[(cities_df["Max Temperature (F)"]>=65) & \
                                (cities_df["Max Temperature (F)"]<=85) & \
                                (cities_df["Wind Speed (mph)"]<=10) & \
                                (cities_df["Cloudiness (%)"]<=0)]

## CSV has am "unnamed" column, is it the index from WeatherPy output? 

# ideal_cities_df.count()
ideal_cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
4,4,Ponta do Sol,PT,32.6667,-17.1000,79.12,34,0,2.86,2021-10-18
16,16,Port-de-Bouc,FR,43.4000,4.9833,66.78,75,0,6.91,2021-10-18
71,71,Santiago,CL,-33.4569,-70.6483,78.24,14,0,3.44,2021-10-18
94,94,Khorramshahr,IR,30.4397,48.1664,80.56,29,0,4.61,2021-10-18
107,107,Chimbarongo,CL,-34.7000,-71.0500,77.38,27,0,6.69,2021-10-18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_cities_df[["City","Country", "Latitude", "Longitude", \
                            "Max Temperature (F)", "Humidity (%)", \
                            "Cloudiness (%)", "Wind Speed (mph)"]].reset_index(drop = True)

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Ponta do Sol,PT,32.6667,-17.1000,79.12,34,0,2.86,
1,Port-de-Bouc,FR,43.4000,4.9833,66.78,75,0,6.91,
2,Santiago,CL,-33.4569,-70.6483,78.24,14,0,3.44,
3,Khorramshahr,IR,30.4397,48.1664,80.56,29,0,4.61,
4,Chimbarongo,CL,-34.7000,-71.0500,77.38,27,0,6.69,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters to search for hotels with 5000 meters.
params = {
    "keyword": "hotel", 
    "radius": 5000, 
    "types": "lodging",
    "key": g_key
}
          
for index, city in hotel_df.iterrows():
    # geocoordinates from the hotel_df
    lat = city["Latitude"]
    lng = city["Longitude"]
    params["location"] = f"{lat}, {lng}"

    # in case there's no seatch result
    try:

        hotel_response = requests.get(base_url, params = params).json()
        #print(json.dumps(response_json, indent = 4, sort_keys = True))

        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]

    except (KeyError, IndexError):
        
        hotel_df.loc[index, "Hotel Name"] = "No results"

hotel_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Ponta do Sol,PT,32.6667,-17.1000,79.12,34,0,2.86,Hotel do Campo
1,Port-de-Bouc,FR,43.4000,4.9833,66.78,75,0,6.91,Hôtel Saint-Roch
2,Santiago,CL,-33.4569,-70.6483,78.24,14,0,3.44,Mercure Santiago Centro
3,Khorramshahr,IR,30.4397,48.1664,80.56,29,0,4.61,Green Hotel
4,Chimbarongo,CL,-34.7000,-71.0500,77.38,27,0,6.69,No results


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))